<h1>1. Import Libraries</h1>

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds

<h1>2. Read Data </h1>

In [2]:
df_distance = pd.read_csv('DistanceMatrix.csv')
df_distance

,Unnamed: 0,DNode0,DNode1,DNode2,DNode3,DNode4,DNode5,DNode6,TNode0,TNode1,ONode0,ONode1,ONode2,ONode3
0,DNode0,0.000000,23.925563,16.308526,14.097823,18.187310,18.844048,38.441819,13.112935,7.513841,10.834469,34.549329,25.511757,20.510295
1,DNode1,23.925563,0.000000,36.209337,28.332810,10.934797,5.081515,14.516256,10.812628,25.879012,13.091094,10.623766,34.334186,13.195232
2,DNode2,16.308526,36.209337,0.000000,7.876527,27.725834,31.127822,50.725593,25.396709,10.330325,23.118243,46.833103,9.203231,32.794069
3,DNode3,14.097823,28.332810,7.876527,0.000000,19.849307,23.251295,42.849066,17.520182,6.583982,15.241716,38.956576,11.413934,24.917542
4,DNode4,18.187310,10.934797,27.725834,19.849307,0.000000,12.952752,22.999759,7.474905,17.395509,16.148761,19.107269,25.850683,5.068235
5,DNode5,18.844048,5.081515,31.127822,23.251295,12.952752,0.000000,19.597771,5.731113,20.797497,8.009579,15.705281,29.252671,15.213187
6,DNode6,38.441819,14.516256,50.725593,42.849066,22.999759,19.597771,0.000000,25.328884,40.395268,27.607350,3.892490,48.850442,17.931524
7,TNode0,13.112935,10.812628,25.396709,17.520182,7.474905,5.731113,25.328884,0.000000,15.066384,8.673856,21.436394,23.521558,9.735340
8,TNode1,7.513841,25.879012,10.330325,6.583982,17.395509,20.797497,40.395268,15.066384,0.000000,12.787918,36.502778,17.997916,22.463744
9,ONode0,10.834469,13.091094,23.118243,15.241716,16.148761,8.009579,27.607350,8.673856,12.787918,0.000000,23.714860,23.473208,18.409196


In [3]:
df_demand = pd.read_csv('DC_demand.csv')
df_demand

,DCLable,Mean,Sigma
0,0,8451.449315,428.374914
1,1,14416.054795,711.954732
2,2,7272.235616,365.361652
3,3,15888.717808,770.001912
4,4,12524.410959,629.789221
5,5,16623.356164,824.405929
6,6,5374.391781,277.091631


In [4]:
dot = df_distance.iloc[9:,8:10].values   # distance between o-nodes and t-nodes (4*2 matrix)
dow = df_distance.iloc[9:,1:8].values    # distance between o-nodes and c-nodes (4*7 matrix)
dtw = df_distance.iloc[7:9,1:8].values   # distance between t-nodes and c-nodes (2*7 matrix)

pi = 1000

# Demand at the warehouses are Normally distributed
mean_w = df_demand['Mean'].values    # Meand demand at the warehouses
std_w = df_demand['Sigma'].values    # Std of demand at the warehouse

<h1>3. Develop Phase-1 model</h1>

In [5]:
# funtion to calculate distrution cost
def cost_dist(x):
    xot = x[:8].reshape(4,2)
    yow = x[8:36].reshape(4,7)
    ztw = x[36:].reshape(2,7)
    cost_ot = np.sum((1/1000)*(xot*(100 + dot*2)))
    cost_ow = np.sum((1/1000)*(yow*(100 + dow*2)))
    cost_tw = np.sum((1/1000)*(ztw*(100 + dtw*2)))
    
    cost = cost_ot + cost_ow + cost_tw
    return cost

In [6]:
# function to calculate soct of missidng demand
def cost_missdem(x):
    yow = x[8:36].reshape(4,7)
    ztw = x[36:].reshape(2,7)
    pack_rcvd = np.sum(yow.T, axis = 1) + np.sum(ztw.T, axis = 1)
    missed_dem = []
    for i in range(len(pack_rcvd)):
        missed_dem.append(quad(lambda x: (x-pack_rcvd[i])*norm(mean_w[i],std_w[i]).pdf(x),pack_rcvd[i],100000))
        list_missed_dem = [list(elem)[0] for elem in missed_dem]
    
    return pi*sum(list_missed_dem)

In [7]:
# function to calculate cost of inequity
def cost_equity(x):
    yow = x[8:36].reshape(4,7)
    ztw = x[36:].reshape(2,7)
    pack_rcvd = np.sum(yow.T, axis = 1) + np.sum(ztw.T, axis = 1)
    equidist = np.sum(pack_rcvd) * (mean_w/sum(mean_w))
    cost = np.sum((pack_rcvd - equidist)**2)
    
    return cost

In [8]:
def cost(x):
    distribution = cost_dist(x)
    misseddemand = cost_missdem(x)
    inequity = cost_equity(x)
    return (distribution + misseddemand + inequity)

<h1>4. Solve Phase-1 model </h1>

In [9]:
# Defines bounds for the variables; Non-negativity
bounds = Bounds(np.zeros(50), np.array(np.ones(50) * np.inf))

In [10]:
# Define constraint: t-nodes cannot ship more than their total packages received from the o-nodes
A = np.zeros((2,50))
A[0,:][[0,2,4,6]] = 1
A[1,:][[1,3,5,7]] = 1
A[0,:][36:43] = -1
A[1,:][43:] = -1

linear_constraint = LinearConstraint(A, [-np.inf, -np.inf], [0, 0])

In [11]:
# Develop the optimization model and solve
res = minimize(cost, np.zeros(50), method='trust-constr', constraints = linear_constraint, 
                    options={'gtol': 1, 'xtol': 10,'disp': True}, bounds=bounds)


`gtol` termination condition is satisfied.
Number of iterations: 16, function evaluations: 714, CG iterations: 56, optimality: 5.67e-01, constraint violation: 0.00e+00, execution time: 1.7e+03 s.


<h1>5. Results </h1>

In [12]:
results = res.x

In [13]:
X_ot = results[:8].reshape(4,2)
Y_ow = results[8:36].reshape(4,7)
Z_tw = results[36:].reshape(2,7)

In [14]:
print("O-nodes to t-nodes:")
print(X_ot)
print("O-nodes to w-node:")
print(Y_ow)
print("t-nodes to w-nodes:")
print(Z_tw)

O-nodes to t-nodes:
[[1050.06495258 1050.04980877]
 [1050.0181492  1049.96283435]
 [1050.01048504 1050.03074739]
 [1050.06104922 1050.01437583]]
O-nodes to w-node:
[[1822.73497383 2982.59642596 1593.45049322 3265.41782475 2618.75022597
  3413.70407168 1221.44052054]
 [1822.76540421 2982.61360793 1593.63419474 3265.44913866 2618.7351237
  3413.82034473 1221.25179092]
 [1822.65284362 2982.6343564  1593.37586486 3265.44364769 2618.69759237
  3413.89186369 1221.4823354 ]
 [1822.68049233 2982.59607862 1593.56009107 3265.37062873 2618.66240867
  3413.82212779 1221.34067668]]
t-nodes to w-nodes:
[[1301.92460883 2473.43066508 1070.06652425 2769.98581163 2093.95777714
  2903.07847098  703.23520485]
 [1301.95517371 2473.57859663 1069.91307699 2769.80933522 2094.12994774
  2903.21906615  703.15276864]]


In [15]:
LTL_ot = np.remainder(X_ot,1000)
LTL_ow = np.remainder(Y_ow,1000)
LTL_tw = np.remainder(Z_tw,1000)

In [16]:
VRPinput = np.zeros([13,13])

VRPinput[0:4,4:6] = LTL_ot.round(0)
VRPinput[0:4,6:] = LTL_ow.round(0)
VRPinput[4:6,6:] = LTL_tw.round(0)

In [17]:
onodes = df_distance.columns[1:].tolist()[9:]
tnodes = df_distance.columns[1:].tolist()[7:9]
dnodes = df_distance.columns[1:].tolist()[:7]

In [18]:
lst1 = []
lst2 = []
lst3 = []

for i in range(len(onodes)):
    for j in range(len(tnodes)):
        lst1.append([onodes[i],tnodes[j]])
        
for i in range(len(onodes)):
    for j in range(len(dnodes)):
        lst2.append([onodes[i],dnodes[j]])
        
for i in range(len(tnodes)):
    for j in range(len(dnodes)):
        lst3.append([tnodes[i],dnodes[j]])
        
lst_node = lst1 + lst2 + lst3

In [20]:
ntruck_ot = np.divmod(X_ot,1000)[0]
ntruck_ow = np.divmod(Y_ow,1000)[0]
ntruck_tw = np.divmod(Z_tw,1000)[0]

In [21]:
pd.DataFrame(ntruck_ot).to_csv("ntruck_ot.csv")
pd.DataFrame(ntruck_ow).to_csv("ntruck_ow.csv")
pd.DataFrame(ntruck_tw).to_csv("ntruck_tw.csv")


In [22]:
pd.DataFrame(ntruck_ot*1000).to_csv("npackages_ot.csv")
pd.DataFrame(ntruck_ow*1000).to_csv("npackages_ow.csv")
pd.DataFrame(ntruck_tw*1000).to_csv("npackages_tw.csv")

In [23]:
pd.DataFrame(ntruck_ot+1).to_csv("wc_ntruck_ot.csv")
pd.DataFrame(ntruck_ow+1).to_csv("wc_ntruck_ow.csv")
pd.DataFrame(ntruck_tw+1).to_csv("wc_ntruck_tw.csv")

In [24]:
pd.DataFrame(X_ot.round(0)).to_csv("wc_npackages_ot.csv")
pd.DataFrame(Y_ow.round(0)).to_csv("wc_npackages_ow.csv")
pd.DataFrame(Z_tw.round(0)).to_csv("wc_npackages_tw.csv")

In [25]:
cost_missdem(results)

313.81629226756434

In [26]:
cost_equity(results)

0.120321674057621

In [27]:
# funtion to calculate worst case truck cost with round-up figures from phase 1
def wc_cost_truck(x):
    xot = x[:8].reshape(4,2)
    yow = x[8:36].reshape(4,7)
    ztw = x[36:].reshape(2,7)
    ntruck_ot = np.divmod(xot.round(0),1000)[0]
    ntruck_ow = np.divmod(yow.round(0),1000)[0]
    ntruck_tw = np.divmod(ztw.round(0),1000)[0]
    
    cost_ot = np.sum(((ntruck_ot+1)*(100 + dot*2)))
    cost_ow = np.sum(((ntruck_ow+1)*(100 + dow*2)))
    cost_tw = np.sum(((ntruck_tw+1)*(100 + dtw*2)))
    
    cost_wc = cost_ot + cost_ow + cost_tw
    return cost_wc

In [28]:
FTL_output = pd.DataFrame(columns = ['From','To','Distance','NumberofTrucks','NumberofPackages'])
FTL_output['From']  = [i[0] for i in lst_node]
FTL_output['To']  = [i[1] for i in lst_node]

disti = []
df_disti = df_distance.copy()
df_disti = df_disti.set_index('Unnamed: 0')


for i in range(len(FTL_output)):
    disti.append(df_disti.loc[FTL_output['From'][i],FTL_output['To'][i]])
    
FTL_output['Distance'] = disti
FTL_output['NumberofTrucks'] = np.concatenate([np.array(ntruck_ot).flatten(),np.array(ntruck_ow).flatten(),np.array(ntruck_tw).flatten()]).tolist()
FTL_output['NumberofPackages'] = np.concatenate([np.array(ntruck_ot*1000).flatten(),np.array(ntruck_ow*1000).flatten(),np.array(ntruck_tw*1000).flatten()]).tolist()

        

In [29]:
pd.DataFrame(FTL_output).to_csv("FTL_output.csv")

In [30]:
FTL_WC_output = pd.DataFrame(columns = ['From','To','Distance','NumberofTrucks','NumberofPackages'])
FTL_WC_output['From']  = [i[0] for i in lst_node]
FTL_WC_output['To']  = [i[1] for i in lst_node]

disti = []
df_disti = df_distance.copy()
df_disti = df_disti.set_index('Unnamed: 0')


for i in range(len(FTL_output)):
    disti.append(df_disti.loc[FTL_output['From'][i],FTL_output['To'][i]])
    
FTL_WC_output['Distance'] = disti
FTL_WC_output['NumberofTrucks'] = np.concatenate([np.array(ntruck_ot+1).flatten(),np.array(ntruck_ow+1).flatten(),np.array(ntruck_tw+1).flatten()]).tolist()
FTL_WC_output['NumberofPackages'] = np.concatenate([np.array(X_ot).flatten(),np.array(Y_ow).flatten(),np.array(Z_tw).flatten()]).tolist()


In [31]:
pd.DataFrame(VRPinput).to_csv("VRPinput.csv")

In [32]:
pd.DataFrame(FTL_WC_output).to_csv("FTL_WC_output.csv")

In [33]:
MaxCostTruck = wc_cost_truck(results)
MaxCostTruck

17943.25498

In [34]:
resulst_Ph1 = np.concatenate([np.array(ntruck_ot*1000).flatten(),np.array(ntruck_ow*1000).flatten(),np.array(ntruck_tw*1000).flatten()])
cost_dist_Ph1 = cost_dist(resulst_Ph1)
cost_dist_Ph1

10972.938364

In [35]:
df_Ph2 = pd.read_csv('VRPoutput.csv')
df_Ph2.head()

,TruckPerRoute,PackageNumber,DistancePerRoute,CostPerRoute
0,2,1142.104537,60.862063,443.448252
1,2,1066.666229,67.080333,468.321332
2,1,840.003085,38.956576,177.913152
3,2,1049.081084,46.635364,386.541456
4,1,890.255699,37.294579,174.589158


In [36]:
cost_dist_Ph2 = df_Ph2['CostPerRoute'].sum()
cost_dist_Ph2

4769.151633999991

In [37]:
MinCostTruck = cost_dist_Ph1 + cost_dist_Ph2
MinCostTruck

15742.08999799999

In [38]:
TotSavings = MaxCostTruck - MinCostTruck
PctSavings = TotSavings/MaxCostTruck
print("Two-phase model will save {} dollars ({} percent) in distribution cost compared to round-up phase 1 results.". format(round(TotSavings,0), round(PctSavings,2)*100))

Two-phase model will save 2201.0 dollars (12.0 percent) in distribution cost compared to round-up phase 1 results.
